In [19]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import yaml
import os
import glob

In [20]:
CONFIG_PATH = "/home/office5/ATE/Ketan_sir/"
def load_config(config_name):
    with open(os.path.join(CONFIG_PATH, config_name)) as file:
        config = yaml.safe_load(file)
    return config

config = load_config("config.yaml")
config

{'l_thresh': 30,
 'u_thresh': 245,
 't_lower': 100,
 't_upper': 200,
 'L2Gradient': True,
 'connectivity': 4,
 'get_pixel_img1': 5,
 'get_pixel_img2': 10,
 'get_kernel_size_value': 1,
 'debug': False,
 'y_size': 200,
 'x_size': 500}

## Get crop portion of image

In [23]:
def show(img):
    plt.figure(figsize = (15,12))
    plt.imshow(img)
    plt.show()

In [24]:
def get_edge_image(img_path):
    # Assume the image come is cropped part
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray,30,255,cv2.THRESH_BINARY)[1]
    edge = cv2.Canny(thresh, config['t_lower'], config['t_upper'], L2gradient = config['L2Gradient'])
    return edge,gray

def get_connected_comp_img(edge):
    lst_component = []
    output = cv2.connectedComponentsWithStats(edge, config['connectivity'], cv2.CV_32S)
    (numLabels, labels, stats, centroids) = output

    # loop over the number of unique connected component labels
    for i in range(0, numLabels):
        if i == 0:
            text = "examining component {}/{} (background)".format(
                i + 1, numLabels)
        # otherwise, we are examining an actual connected component
        else:
            text = "examining component {}/{}".format( i + 1, numLabels)
        # x_ = stats[i, cv2.CC_STAT_LEFT]
        # y_ = stats[i, cv2.CC_STAT_TOP]
        w_ = stats[i, cv2.CC_STAT_WIDTH]
        h_ = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]
        (cX, cY) = centroids[i]

        keepWidth = w_ > 2
        keepHeight = h_ > 2
        keepArea = area > 50

        if all((keepWidth, keepHeight, keepArea)):
            componentMask = (labels == i).astype("uint8") * 255
            # print(componentMask)
            # cv2.imwrite(f"/home/office5/Image_Code/Ketan_sir/try1_fun_component/{i}.png",componentMask)
            color_img  = cv2.cvtColor(componentMask,cv2.COLOR_GRAY2BGR)
            lst_component.append(color_img)
    return lst_component

In [25]:
def get_extreme_point(all_component_image):

    # to store all extreme point
    extr_point = []


    # Here define range from 1 to n because 0 image is backgroumd
    for i in range(1,len(all_component_image)):
        gray = cv2.cvtColor(all_component_image[i],cv2.COLOR_BGR2GRAY)
        coords = np.column_stack(np.where(gray == 255))

        # print(coords)
        a = np.min(coords,axis=0)
        b = np.max(coords,axis=0)

        y_min,x_min,y_max,x_max = a[0],a[1],b[0],b[1]

        points = []

        # for i in range(len(coords)):
        #     if(y_min == coords[i][0]):
        #         pass
        #     else:
        #         points.append([coords[i-1][0], coords[i-1][1]]) 
        #         break
            
        for i in range(len(coords)):
            if(x_min == coords[i][1]):
                points.append([coords[i][0], coords[i][1]]) 
                break
            else:
                pass
        points.append([coords[-1][0] , coords[-1][1]])
        points = np.array(points)

        for i in range(3):
            extr_point.append([points[i][0], points[i][1]])

    return extr_point
        


In [26]:
# def get_index_kernel(ori_gray,arr2,def_arr):

#     y1_s = arr2[0] + 2744 - config['top_left_point_kernel']
#     y2_s = arr2[0] + 2744 + config['bottom_right_point_kernel']
#     x1_s = arr2[1] + 1824 - config['top_left_point_kernel']
#     x2_s = arr2[1] + 1824 + config['bottom_right_point_kernel']
#     # print("gr",gray)
#     kernel_value =  ori_gray[y1_s:y2_s,x1_s:x2_s]
#     print("Kerenl_value", kernel_value)
#     count = 0
#     for i in range(len(kernel_value[0])):
#         for j in range(len(kernel_value[1])):
#             def_arr[i][j] = (np.subtract(int(kernel_value[config['top_left_point_kernel']][config['top_left_point_kernel']]),int(kernel_value[i][j])))
#             def_arr[config['top_left_point_kernel']][config['top_left_point_kernel']] = 5 
#             if(def_arr[i][j]==0):
#                 def_arr[i][j] = 1
#             elif(def_arr[i][j]<0):
#                 def_arr[i][j] = 0
#             else:
#                 def_arr[i][j] = 1
            
#     return def_arr

# trial only for part of image

In [27]:
def get_index_kernel(ori_gray,extreme_point_arr,set_kernel_value):

    # y1_s = extreme_point_arr[0] - config['top_left_point_kernel']
    # y2_s = extreme_point_arr[0] + config['bottom_right_point_kernel']
    # x1_s = extreme_point_arr[1] - config['top_left_point_kernel']
    # x2_s = extreme_point_arr[1] + config['bottom_right_point_kernel']

    y1_s = extreme_point_arr[0] - config['get_kernel_size_value']
    y2_s = extreme_point_arr[0] + config['get_kernel_size_value']
    x1_s = extreme_point_arr[1] - config['get_kernel_size_value']
    x2_s = extreme_point_arr[1] + config['get_kernel_size_value']

    try:
        kernel_value =  ori_gray[y1_s:y2_s + 1,x1_s:x2_s + 1]
        # print("Kernel_value -------------------------")
        # print(kernel_value)
        
        for i in range(len(kernel_value[0])):
            for j in range(len(kernel_value[1])):
                set_kernel_value[i][j] = (np.subtract(int(kernel_value[config['get_kernel_size_value']][config['get_kernel_size_value']]),int(kernel_value[i][j])))
                set_kernel_value[config['get_kernel_size_value']][config['get_kernel_size_value']] = 5 
                if(set_kernel_value[i][j]==0):
                    set_kernel_value[i][j] = 1
                elif(set_kernel_value[i][j]<0):
                    set_kernel_value[i][j] = 0
                else:
                    set_kernel_value[i][j] = 1
    except:
        # print(kernel_value)
        set_kernel_value = np.zeros((3,3))
        # print(set_kernel_value)
    return set_kernel_value

In [28]:
def get_removed_extreme_point(ori_gray,all_extreme_point):

    extreme_point_list = []
    dict1 = {}
    kernel_shape = (config['get_kernel_size_value'] * 2) + 1 ## 1->3 2->5 3->7 kerenl_size
    m = 0
    dict1 = {}
    print("all point testing Point")
    for i in range(len(all_extreme_point)):
        extreme_point_arr = np.array(all_extreme_point[i])
        # print(arr)
        zero_kernel_arr = np.zeros((kernel_shape,kernel_shape))
        kernel_value = get_index_kernel(ori_gray,extreme_point_arr,zero_kernel_arr)
        unique_1_0 = np.unique(kernel_value,return_counts=True)
        # print(unique_1_0)
        if len(unique_1_0[0])==3:
            count_0 = unique_1_0[1][0]
            count_1 = unique_1_0[1][1]
            # print(count_0,count_1)
            dict1['0'] = count_0
            dict1['1'] = count_1
            if (count_0==3 and count_1==5) or (count_0==5 and count_1==3):
                temp_kernel_size_value = config['get_kernel_size_value']
                corner_1 = kernel_value[0][0]
                corner_2 = kernel_value[0][kernel_shape-1]
                corner_3 = kernel_value[kernel_shape-1][0]
                corner_4 = kernel_value[kernel_shape-1][kernel_shape-1]

                corner_point = np.array([corner_1,corner_2,corner_3,corner_4])
                for key, value in dict1.items():
                    if value == 3:
                        corner_index = np.where(corner_point==int(key)) 

                        if corner_index[0][0] == 0:
                            x,y = 0,0
                            if (kernel_value[x][y]==int(key) and kernel_value[x+1][y]==int(key) and kernel_value[x][y+1]==int(key)):
                                # print('--------0',[a[i][0],a[i][1]])
                                extreme_point_list.append([all_extreme_point[i][0],all_extreme_point[i][1]])

                        if corner_index[0][0] == 1:
                            x,y = 0,kernel_shape-1
                            if (kernel_value[x][y]==int(key) and kernel_value[x][y-1]==int(key) and kernel_value[x+1][y]==int(key)):
                                # print('--------1',[a[i][0],a[i][1]])
                                extreme_point_list.append([all_extreme_point[i][0],all_extreme_point[i][1]])

                        if corner_index[0][0] == 2:
                            x,y = kernel_shape-1,0
                            if (kernel_value[x][y]==int(key) and kernel_value[x-1][y]==int(key) and kernel_value[x][y+1]==int(key)):
                                # print('--------2',[a[i][0],a[i][1]])
                                extreme_point_list.append([all_extreme_point[i][0],all_extreme_point[i][1]])

                        if corner_index[0][0] == 3:
                            x,y = kernel_shape-1,kernel_shape-1
                            if (kernel_value[x][y]==int(key) and kernel_value[x][y-1]==int(key) and kernel_value[x-1][y]==int(key)):
                                # print('--------3',[a[i][0],a[i][1]])
                                extreme_point_list.append([all_extreme_point[i][0],all_extreme_point[i][1]])


    return extreme_point_list

# a = np.load('coordinate.npy')
# a = [tuple(row) for row in a]
# a = np.unique(a,axis=0)
# h = config['top_left_point_kernel'] + config['bottom_right_point_kernel']

# def_arr = np.zeros((h,h))
# dict1 = {}
# extrmem_point_list = []
# value = get_extreme_point(a,dict1,extrmem_point_list)
# print(value)

In [29]:
def main(img_path):
    edge,original_gray = get_edge_image(img_path)
    all_component_image = get_connected_comp_img(edge)
    all_extreme_point =  get_extreme_point(all_component_image)
    remove_duplicate_extreme_point = [tuple(row) for row in all_extreme_point]
    remove_duplicate_extreme_point = np.unique(remove_duplicate_extreme_point,axis=0)
    final_extreme_point = get_removed_extreme_point(original_gray,remove_duplicate_extreme_point)
    return final_extreme_point

img_path = '/home/office5/Downloads/01.png'
final_extreme_point = main(img_path)
print("*"*80)
print(final_extreme_point)

[ WARN:0@306.188] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/home/office5/Downloads/01.png'): can't open/read file: check file path/integrity


error: OpenCV(4.6.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


### 2nd function
making function to get point from original_image, defective image

In [35]:
def get_point_on_image(final_extreme_point,anna2_gray_img,anna3_gray_img):
    second_img_point = []
    print("anna2_gray_img",anna2_gray_img.shape)
    print("anna3_gray_img",anna3_gray_img.shape)

    for i in range(len(final_extreme_point)):
        ori_img_loc_x = final_extreme_point[i][1]
        ori_img_loc_y = final_extreme_point[i][0]
        x1_f = ori_img_loc_x - config['get_pixel_img1']
        x2_f = ori_img_loc_x + config['get_pixel_img1']
        y1_f = ori_img_loc_y - config['get_pixel_img1']
        y2_f = ori_img_loc_y + config['get_pixel_img1']

        img_crop1  = anna2_gray_img[y1_f:y2_f,x1_f:x2_f]
        img_first = img_crop1.astype(np.int8)
        # print("Original",ori_img_loc_y,ori_img_loc_x)

        x1_s = ori_img_loc_x - config['get_pixel_img2']
        x2_s = ori_img_loc_x + config['get_pixel_img2']
        y1_s = ori_img_loc_y - config['get_pixel_img2']
        y2_s = ori_img_loc_y + config['get_pixel_img2']

        img_crop2  = anna3_gray_img[y1_s:y2_s,x1_s:x2_s]
        img_snd = img_crop2.astype(np.int8)

        dict1 = {}
        for i in range(10):
            for j in range(10):
                dict1[i,j] = np.mean(np.abs(img_first  - img_snd[i:i+10,j:j+10]))
        i,j = min(dict1, key=dict1.get)

        # print(i,j)
        s_img_extr_point_x = x1_s + config['get_pixel_img2'] - config['get_pixel_img1'] + j # 10 for each block of first image and -5 for adding from width and height of first image
        s_img_extr_point_y =y1_s + config['get_pixel_img2'] - config['get_pixel_img1'] + i

        second_img_point.append([s_img_extr_point_y,s_img_extr_point_x])
        # print("From second image",s_img_extr_point_y,s_img_extr_point_x)

    for i in range(len(final_extreme_point)):
        anna2_gray_img[final_extreme_point[i][0]][final_extreme_point[i][1]] = 255

    for i in range(len(second_img_point)):
        anna3_gray_img[second_img_point[i][0]][second_img_point[i][1]] = 255

    return second_img_point,anna2_gray_img,anna3_gray_img

anna2_img = "/home/office5/Image_Code/Ketan_sir/testing_images/Anna2.png"
anna2_gray_img = cv2.imread(anna2_img,0)

anna3_img = "/home/office5/Image_Code/Ketan_sir/testing_images/Anna3.png"
anna3_gray_img = cv2.imread(anna3_img,0)

second_img_point,img1_point, img2_point = get_point_on_image(final_extreme_point,anna2_gray_img,anna3_gray_img)

# plt.imshow(img1_point)
# plt.show()
# plt.imshow(img2_point)
# plt.show()

# cv2.imwrite('/home/office5/Image_Code/Ketan_sir/testing_images/anna7_redpoint.png',img1_point)
# cv2.imwrite('/home/office5/Image_Code/Ketan_sir/testing_images/anna8_redpoint.png',img2_point)

anna2_gray_img (392, 912)
anna3_gray_img (392, 912)


In [32]:
print(final_extreme_point)
print(second_img_point)

[[12, 435], [25, 440], [70, 406], [77, 234], [85, 497], [106, 42], [113, 453], [129, 162], [216, 248], [231, 728], [231, 735], [232, 667], [233, 266], [237, 218], [240, 240], [240, 429], [242, 429], [257, 407], [262, 612], [262, 728], [262, 735], [265, 671], [283, 761], [285, 495], [286, 413], [287, 212], [287, 298], [333, 488], [340, 867], [363, 871], [370, 509], [370, 531], [371, 400], [375, 574]]
[[12, 435], [24, 440], [69, 406], [77, 234], [85, 497], [105, 43], [113, 453], [130, 163], [216, 248], [231, 728], [231, 735], [232, 667], [233, 266], [237, 219], [240, 240], [241, 429], [243, 429], [257, 407], [262, 612], [262, 728], [262, 735], [265, 671], [283, 761], [285, 495], [286, 413], [287, 213], [287, 298], [333, 488], [339, 867], [362, 871], [369, 509], [369, 531], [370, 400], [374, 574]]


In [ ]:
# img_path = '/home/office5/Image_Code/Ketan_sir/gray.png'
# edge,original_gray = get_edge_image(img_path)
# all_component_image = get_connected_comp_img(edge)
# all_extreme_point =  get_extreme_point(all_component_image)
# # print("-"*80)
# # print(all_extreme_point)
# remove_duplicate_extreme_point = [tuple(row) for row in all_extreme_point]
# remove_duplicate_extreme_point = np.unique(remove_duplicate_extreme_point,axis=0)
# # print("$"*30)
# # print(a)
# final_extreme_point = get_removed_extreme_point(original_gray,remove_duplicate_extreme_point)
# print("*"*80)
# print(final_extreme_point)
# # For priting white point on extreme point pixel
# for i in range(len(value)):
#     original_gray[value[i][0]][value[i][1]] = 255
# cv2.imwrite('w.png',original_gray)